In [1]:
# reference example
from nequip.data import dataset_from_config
from nequip.utils import Config
#from nequip.utils.misc import get_default_device_name
#from nequip.utils.config import _GLOBAL_ALL_ASKED_FOR_KEYS

from nequip.model import model_from_config


default_config = dict(
    root="./",
    tensorboard=False,
    wandb=False,
    model_builders=[
        "SimpleIrrepsConfig",
        "EnergyModel",
        "PerSpeciesRescale",
        "StressForceOutput",
        "RescaleEnergyEtc",
    ],
    dataset_statistics_stride=1,
    device='cuda',
    default_dtype="float64",
    model_dtype="float32",
    allow_tf32=True,
    verbose="INFO",
    model_debug_mode=False,
    equivariance_test=False,
    grad_anomaly_mode=False,
    gpu_oom_offload=False,
    append=False,
    warn_unused=False,
    _jit_bailout_depth=2,  # avoid 20 iters of pain, see https://github.com/pytorch/pytorch/issues/52286
    # Quote from eelison in PyTorch slack:
    # https://pytorch.slack.com/archives/CDZD1FANA/p1644259272007529?thread_ts=1644064449.039479&cid=CDZD1FANA
    # > Right now the default behavior is to specialize twice on static shapes and then on dynamic shapes.
    # > To reduce warmup time you can do something like setFusionStrartegy({{FusionBehavior::DYNAMIC, 3}})
    # > ... Although we would wouldn't really expect to recompile a dynamic shape fusion in a model,
    # > provided broadcasting patterns remain fixed
    # We default to DYNAMIC alone because the number of edges is always dynamic,
    # even if the number of atoms is fixed:
    _jit_fusion_strategy=[("DYNAMIC", 3)],
    # Due to what appear to be ongoing bugs with nvFuser, we default to NNC (fuser1) for now:
    # TODO: still default to NNC on CPU regardless even if change this for GPU
    # TODO: default for ROCm?
    _jit_fuser="fuser1",
)

# All default_config keys are valid / requested
#_GLOBAL_ALL_ASKED_FOR_KEYS.update(default_config.keys())

In [2]:
config

Available objects for config:
    AliasManager
    DisplayFormatter
    HistoryManager
    IPCompleter
    IPKernelApp
    LoggingMagics
    MagicsManager
    OSMagics
    PrefilterManager
    ScriptMagics
    StoreMagics
    ZMQInteractiveShell


In [3]:
config = Config.from_file('./configs/example_ETN.yaml', defaults=default_config)
    

dataset = dataset_from_config(config, prefix="dataset")

validation_dataset = None

dataset[0]

AtomicData(atom_types=[21, 1], cell=[3, 3], edge_cell_shift=[364, 3], edge_index=[2, 364], forces=[21, 3], pbc=[3], pos=[21, 3], total_energy=[1])

In [4]:
# Trainer
from nequip.train.trainer import Trainer
from e3nn import o3

trainer = Trainer(model=None, **Config.as_dict(config))

# what is this
# to update wandb data?
config.update(trainer.params)

# = Train/test split =
trainer.set_dataset(dataset, validation_dataset)

#config['model_input_fields'] = {'node_spin': o3.Irreps('1x1e')}
Nc = 10
N_rank_spec = 4
config['Nc'] = Nc
config['N_rank_spec'] = N_rank_spec

# = Build model =
final_model = model_from_config(
    config=config, initialize=True, dataset=trainer.dataset_train
)

DEBUG:root:* Initialize Output
  ...generate file name results/aspirin/example/log
  ...open log file results/aspirin/example/log
  ...generate file name results/aspirin/example/metrics_epoch.csv
  ...open log file results/aspirin/example/metrics_epoch.csv
  ...generate file name results/aspirin/example/metrics_initialization.csv
  ...open log file results/aspirin/example/metrics_initialization.csv
  ...generate file name results/aspirin/example/metrics_batch_train.csv
  ...open log file results/aspirin/example/metrics_batch_train.csv
  ...generate file name results/aspirin/example/metrics_batch_val.csv
  ...open log file results/aspirin/example/metrics_batch_val.csv
  ...generate file name results/aspirin/example/best_model.pth
  ...generate file name results/aspirin/example/last_model.pth
  ...generate file name results/aspirin/example/trainer.pth
  ...generate file name results/aspirin/example/config.yaml
Torch device: cuda
instantiate Loss
...Loss_param = dict(
...   optional_args 

...   positional_args = {'irreps_in': {'pos': 1x1oe, 'edge_index': None, 'edge_types': 1x0ee, 'node_attrs': 3x0ee, 'node_features': 3x0ee, 'edge_embedding': 8x0ee, 'edge_cutoff': 1x0ee, 'edge_attrs': 1x0ee+1x1oe+1x2ee, 'edge_features_F': 10x0ee+10x1oe+10x2ee, 'node_features_F': 10x0ee+10x1oe+10x2ee}})
/Users/temporary/Documents/GitHub/pytorch-intel-mps/torch/jit/_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
instantiate ScalarMLP
        all_args :                               mlp_latent_dimensions <-                     edge_eng_mlp_latent_dimensions
        all_args :                                    mlp_nonlinearity <-                          edge_eng_mlp_nonlinearity
        all_args :                                  mlp_

In [5]:

import torch
from torch.nn.functional import one_hot
from nequip.data import AtomicData, AtomicDataDict
from torch.nn.functional import one_hot
from e3nn.nn import FullyConnectedNet
from allegro import with_edge_spin_length
from allegro import _keys
from torch import nn
import math

trainer.model = final_model
data0 = AtomicData.to_AtomicDataDict(dataset[0])

In [6]:
import torch
from torch.nn.functional import one_hot
from nequip.data import AtomicData, AtomicDataDict
from torch.nn.functional import one_hot
from e3nn.nn import FullyConnectedNet
    
from torch import nn
import math

data_new = final_model(data0)

In [7]:
data_new['edge_features_F'].shape

torch.Size([364, 9, 10])

In [8]:
data_new['node_features_F'].shape

torch.Size([21, 9, 10])

In [65]:

import torch
from torch.nn.functional import one_hot
from nequip.data import AtomicData, AtomicDataDict
from torch.nn.functional import one_hot
from e3nn.nn import FullyConnectedNet
from allegro import with_edge_spin_length
from allegro import _keys
from torch import nn
import math

data = data0


import copy

data_rot = {key: torch.clone(data0[key]) for key in data0}

irreps_sh = o3.Irreps('1x0e + 1x1o + 1x2e') #o3.Irreps.spherical_harmonics(lmax=2)
irreps_sh_r = o3.Irreps('1x1o')

alpha, beta, gamma = o3.rand_angles(100)

rot_matrix = irreps_sh.D_from_angles(alpha[0], beta[0], gamma[0])
rot_matrix_r = irreps_sh_r.D_from_angles(alpha[0], beta[0], gamma[0])


data_rot['pos'] = data_rot['pos'] @ rot_matrix_r

In [78]:
torch.manual_seed(32)

F = final_model(data)['node_features_F']

torch.manual_seed(32)

F_rot =final_model(data_rot)['node_features_F']


F_rot_rot = torch.einsum('Njn,jk->Nkn', F_rot, rot_matrix.T)

if torch.allclose(F, F_rot_rot, atol=1e-05):
    print('F is equivariant')

F is equivariant
